# Lab 1

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
#Test
df = pd.read_csv('ds_salary.csv')

## 1. General Salary Data
Start by plotting and describing the general shape of the salary data. This initial
step sets the foundation for your analysis.

,...1,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
576,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
577,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
578,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
579,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


## 2. Effect of Experience on Salary
Investigate how experience influences data scientist salaries. Create visualizations or
tables to illustrate any trends or patterns.

## 3. Impact of Company Size on Salary
Explore how company size affects data scientist salaries. Generate plots or tables to
visualize this relationship.

## 4. Job Title Analysis
Job titles can vary significantly. For this project, select a keyword or phrase to distinguish
between different job titles. For instance, you can differentiate between job
titles containing the word ”Analyst” and those that do not. To achieve this, you may
find the str.contains function in Pandas helpful. Refer to this resource for examples:
https://www.geeksforgeeks.org/python-pandas-series-str-contains. Explore
various job titles to decide on the keyword or phrase you want to focus on.

## 5. Interactions Between Relationships
Investigate potential interactions between the three factors (experience, company size,
and job title). Create a two-way table that displays essential statistics at the intersection
of two variables. For instance, construct a table with job titles on the left,
company sizes at the top, and each cell representing the mean salary for each job
title/company size combination.

## 6. Conclusion
Summarize your findings by listing all significant relationships you have discovered
during your analysis.